## Song Recommendation

After my data pre-processing and clustering using gaussian mixture model (GMM) to find the relevant cluster with most reggaeton. I went ahead and built my recommendation algorithm. I tried eucledian distance and Matrix Factorization and found that matrix factorization gave a better result to my particular liking, and it also incluided actual reggaeton songs.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA


from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [6]:
clustered_spotify_df = pd.read_csv("../../Reggaeton_Recommender_System/spotify-datasets/kaggle-yamaerenay-dataset/clustered_spotify_df.csv")
clean_spotify_df = pd.read_csv("../spotify-datasets/kaggle-yamaerenay-dataset/tracks_JoseCleaned.csv")

In [5]:
clustered_spotify_df

,duration_ms,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,time_signature_1,time_signature_3,time_signature_4,time_signature_5,era_FLAG_1990s,era_FLAG_2000s,era_FLAG_2010s,genres_reggaeton,GMM_cluster,KMeans_cluster
0,160187,0,0.484,0.2650,-11.101,1,0.0322,0.394,0.000000,0.1490,...,False,True,False,False,False,True,False,False,11,15
1,141987,0,0.405,0.3650,-10.226,0,0.0289,0.255,0.000005,0.1630,...,False,False,True,False,False,True,False,False,6,0
2,157093,0,0.477,0.3520,-14.165,1,0.0300,0.406,0.000000,0.1220,...,False,False,True,False,False,True,False,False,15,9
3,114987,0,0.617,0.7110,-6.433,1,0.0297,0.360,0.000002,0.0841,...,False,False,True,False,False,True,False,False,10,5
4,115240,0,0.409,0.6390,-7.628,1,0.0302,0.288,0.000000,0.3430,...,False,False,True,False,False,True,False,False,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293122,265846,0,0.528,0.6730,-3.639,1,0.0314,0.143,0.000000,0.0989,...,False,False,True,False,False,False,True,False,11,2
293123,82500,0,0.609,0.0172,-28.573,1,0.1180,0.996,0.973000,0.1080,...,False,False,True,False,False,False,True,False,12,3
293124,185250,0,0.562,0.0331,-25.551,1,0.1030,0.996,0.961000,0.1110,...,False,True,False,False,False,False,True,False,15,15
293125,258267,0,0.560,0.5180,-7.471,0,0.0292,0.785,0.000000,0.0648,...,False,False,True,False,False,False,True,False,11,3


In [9]:
clean_spotify_df


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,valence,tempo,time_signature,track_age_FLAG,era_FLAG,followers,genres,artist_name,artists_popularity,GMM_cluster
0,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,['Gerry & The Pacemakers'],3UmBeGyNwr4iDWi1vTxWi8,2008,0.484,0.2650,...,0.285,113.564,3,13,2000s,175360.0,"['adult standards', 'brill building pop', 'bri...",Gerry & The Pacemakers,55,11
1,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,['Gerry & The Pacemakers'],3UmBeGyNwr4iDWi1vTxWi8,2008,0.405,0.3650,...,0.588,104.536,4,13,2000s,175360.0,"['adult standards', 'brill building pop', 'bri...",Gerry & The Pacemakers,55,6
2,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,['Gerry & The Pacemakers'],3UmBeGyNwr4iDWi1vTxWi8,2008,0.477,0.3520,...,0.478,106.773,4,13,2000s,175360.0,"['adult standards', 'brill building pop', 'bri...",Gerry & The Pacemakers,55,15
3,7LfvdUcwrrMKVh8WPGXnbA,How Do You Do It? - Mono,29,114987,0,['Gerry & The Pacemakers'],3UmBeGyNwr4iDWi1vTxWi8,2008,0.617,0.7110,...,0.963,142.266,4,13,2000s,175360.0,"['adult standards', 'brill building pop', 'bri...",Gerry & The Pacemakers,55,10
4,25qyOLQyX7bsceW3UL55BW,Hello Little Girl,25,115240,0,['Gerry & The Pacemakers'],3UmBeGyNwr4iDWi1vTxWi8,2008,0.409,0.6390,...,0.928,145.903,4,13,2000s,175360.0,"['adult standards', 'brill building pop', 'bri...",Gerry & The Pacemakers,55,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293122,0MmaEacabpK8Yp3Mdeo5uY,下雨天,50,265846,0,['芝麻'],5VGgFE9nPgMfEnYiPT5J2B,2020,0.528,0.6730,...,0.297,130.066,4,1,2010s,929.0,['chinese viral pop'],芝麻,36,11
293123,1dKxf4Ht2SsKLyXfSDJAgy,The Cutest Puppy,67,82500,0,['Laureen Conrad'],7vgGpuiXdNlCmc994PlMlz,2020,0.609,0.0172,...,0.890,68.619,4,1,2010s,23.0,['instrumental lullaby'],Laureen Conrad,52,12
293124,0SjsIzJkZfDU7wlcdklEFR,John Brown's Song,66,185250,0,['Gregory Oberle'],4MxqhahGRT4BPz1PilXGeu,2020,0.562,0.0331,...,0.386,63.696,3,1,2010s,91.0,['instrumental lullaby'],Gregory Oberle,55,15
293125,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],1QLBXKM5GCpyQQSVMNZqrZ,2020,0.560,0.5180,...,0.211,131.896,4,1,2010s,896.0,['chinese viral pop'],阿YueYue,38,11


In [10]:
# add clusters and reggaeton labels, remove columns I won't need
clean_spotify_df['GMM_cluster'] = clustered_spotify_df['GMM_cluster']
clean_spotify_df['genres_reggaeton'] = clustered_spotify_df['genres_reggaeton']
clean_spotify_df['KMeans_cluster'] = clustered_spotify_df['KMeans_cluster']


In [13]:
# simplify data set for matrix factorization
clean_spotify_df.info() #drop features that won't multiply

cleaner_spotify_df = clean_spotify_df.drop(['explicit', 'artists','id_artists','release_date','key','mode', 'time_signature'], axis=1)
cleaner_spotify_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293127 entries, 0 to 293126
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  293127 non-null  object 
 1   name                293127 non-null  object 
 2   popularity          293127 non-null  int64  
 3   duration_ms         293127 non-null  int64  
 4   explicit            293127 non-null  int64  
 5   artists             293127 non-null  object 
 6   id_artists          293127 non-null  object 
 7   release_date        293127 non-null  int64  
 8   danceability        293127 non-null  float64
 9   energy              293127 non-null  float64
 10  key                 293127 non-null  int64  
 11  loudness            293127 non-null  float64
 12  mode                293127 non-null  int64  
 13  speechiness         293127 non-null  float64
 14  acousticness        293127 non-null  float64
 15  instrumentalness    293127 non-nul

In [26]:
# select only GMM cluster #3
cleanest_spotify_df = cleaner_spotify_df[cleaner_spotify_df['GMM_cluster'] == 3]

In [27]:
cleanest_spotify_df

,id,name,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,tempo,track_age_FLAG,era_FLAG,followers,genres,artist_name,artists_popularity,GMM_cluster,genres_reggaeton,KMeans_cluster
2981,4iozhXt27eMl39W5z7R8H6,Little Red Corvette,51,296907,0.748,0.746,-9.721,0.1110,0.28100,0.000033,...,123.144,39,1980s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,9
3021,04MjaNej8XUHI8jbz2THez,7,41,310467,0.747,0.749,-6.255,0.0563,0.09170,0.000000,...,100.722,29,1990s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,5
3052,7oZKxGPHuMVSVmFzuiKkWG,How Come U Don't Call Me Anymore,40,234773,0.556,0.128,-14.481,0.0688,0.90500,0.004060,...,83.065,28,1990s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,3
3065,5xcb3TD6lZ4X7RId59DNxo,She's Always in My Hair,41,207427,0.729,0.777,-10.157,0.0424,0.08910,0.000647,...,92.308,28,1990s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,5
3066,37boVGhMqQF129fn3jHQU4,Come,35,673160,0.750,0.843,-5.870,0.0798,0.00439,0.012200,...,91.378,27,1990s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292963,0jngaMh4nKBzquXKeaaYzv,Bro,51,168259,0.708,0.622,-8.503,0.3770,0.17800,0.000000,...,173.133,1,2010s,7045.0,"['swedish gangsta rap', 'swedish hip hop', 'sw...",Lelo,46,3,False,5
292973,4UAJQSBKU7ARIuMnkwDp3n,Ahhä,52,172630,0.924,0.488,-7.217,0.1390,0.15500,0.000014,...,100.011,1,2010s,6097.0,"['swedish gangsta rap', 'swedish hip hop', 'sw...",La Rosy,46,3,False,9
292974,5I2QHXyUj8nx1Onu6fXu8C,Distans,51,177267,0.938,0.506,-6.851,0.1720,0.51000,0.000000,...,134.975,1,2010s,3354.0,"['swedish hip hop', 'swedish trap pop']",Sarettii,48,3,False,9
292976,7L9N5BTob9ISdb7IoxV2ZU,miSs U <3,50,202106,0.779,0.700,-7.917,0.0733,0.48400,0.000000,...,93.070,1,2010s,19310.0,"['swedish gangsta rap', 'swedish hip hop']",Alex Ceesay,55,3,False,9


In [29]:
# OMG! pure reggaeton
cleanest_spotify_df[cleanest_spotify_df['genres_reggaeton'] == True]

,id,name,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,tempo,track_age_FLAG,era_FLAG,followers,genres,artist_name,artists_popularity,GMM_cluster,genres_reggaeton,KMeans_cluster
59495,1CxVZqcEpSINoTEeNHYIjW,Me Acuerdo,46,349053,0.579,0.458,-8.522,0.0617,0.174000,0.000000,...,172.171,27,1990s,553171.0,"['latin', 'latin hip hop', 'rap conciencia', '...",Vico C,67,3,True,9
59514,4sPi2oUEFmu7RMtllJr040,La Movida,39,237122,0.822,0.612,-4.660,0.2290,0.138000,0.000000,...,94.062,19,2000s,553171.0,"['latin', 'latin hip hop', 'rap conciencia', '...",Vico C,67,3,True,9
59531,1fK20GB6jode6jIL2Jnyd2,Abusando,37,233960,0.811,0.781,-3.561,0.2380,0.040500,0.000000,...,93.868,19,2000s,553171.0,"['latin', 'latin hip hop', 'rap conciencia', '...",Vico C,67,3,True,9
86893,17PNg7rk7PI0x2VYkOsjS0,Que Pecado,25,224000,0.727,0.644,-6.151,0.4290,0.052000,0.000000,...,104.911,7,2010s,48963.0,"['latin', 'latin hip hop', 'panamanian pop', '...",Latin Fresh,49,3,True,5
97417,0lbvG4WwnSABv6ZKURaKjQ,Un Volvo,32,107729,0.954,0.833,-8.264,0.2060,0.006930,0.000000,...,109.231,22,1990s,67690.0,"['latin', 'latin hip hop', 'panamanian pop', '...",El Chombo,56,3,True,9
97425,08mJYMiKCUgxjABAOUzKWT,El Pelo,28,123115,0.903,0.782,-9.853,0.1190,0.071500,0.018500,...,109.179,22,1990s,67690.0,"['latin', 'latin hip hop', 'panamanian pop', '...",El Chombo,56,3,True,9
97427,7oKxQDxZYbrgeudGtfx562,Speedy Gonzalez al Baile Llego,31,120947,0.883,0.747,-8.190,0.2140,0.008080,0.000000,...,109.162,22,1990s,67690.0,"['latin', 'latin hip hop', 'panamanian pop', '...",El Chombo,56,3,True,5
97429,4eUl4DVViDPgld45VrslQF,La Papa Rusa,25,103393,0.734,0.911,-7.607,0.3340,0.118000,0.000059,...,217.891,22,1990s,67690.0,"['latin', 'latin hip hop', 'panamanian pop', '...",El Chombo,56,3,True,9
97431,1LDnKdnFjZQ67ggh37DFx9,Mueve el Cuerpo y la Cabeza,28,125309,0.941,0.896,-8.035,0.1010,0.013400,0.041900,...,109.246,22,1990s,67690.0,"['latin', 'latin hip hop', 'panamanian pop', '...",El Chombo,56,3,True,9
97432,3ukkJgF1aJOsZ5raKLS2X4,A Que Te Lo Meten...,25,136333,0.905,0.717,-8.730,0.1880,0.000122,0.000124,...,109.247,22,1990s,67690.0,"['latin', 'latin hip hop', 'panamanian pop', '...",El Chombo,56,3,True,9


### Matrix Factorization with my seed songs

In [36]:
user_df = pd.read_csv("../Output/seed_song_features_output.csv", index_col='index' )

In [40]:
user_df['track_id'] = user_df['id']

In [49]:
user_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.0
mean,0.791960,0.791440,5.640000,-6.133160,0.440000,0.118398,0.174887,0.003006,0.155708,0.740500,100.795520,203314.460000,4.0
std,0.089234,0.093965,4.003876,2.047188,0.501427,0.086742,0.150330,0.017123,0.141273,0.145159,23.064468,36171.745417,0.0
min,0.434000,0.532000,0.000000,-13.723000,0.000000,0.036600,0.001920,0.000000,0.037800,0.238000,82.027000,123573.000000,4.0
25%,0.749250,0.736500,1.250000,-7.068500,0.000000,0.062275,0.038750,0.000000,0.069775,0.682000,93.256000,176231.500000,4.0
50%,0.818000,0.816500,5.500000,-5.902000,0.000000,0.102500,0.138500,0.000004,0.100200,0.759500,94.998500,202672.500000,4.0
75%,0.848000,0.856000,10.000000,-5.072750,1.000000,0.140500,0.267750,0.000095,0.210000,0.837000,96.133250,222633.250000,4.0
max,0.911000,0.927000,11.000000,-1.310000,1.000000,0.525000,0.544000,0.121000,0.861000,0.964000,183.985000,298000.000000,4.0


In [42]:
user_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'track_id'],
      dtype='object')

In [51]:
clean_user_df = user_df.drop(['key','mode', 'time_signature', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'track_id'], axis=1)
clean_user_df.info()
# perfect

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      50 non-null     float64
 1   energy            50 non-null     float64
 2   loudness          50 non-null     float64
 3   speechiness       50 non-null     float64
 4   acousticness      50 non-null     float64
 5   instrumentalness  50 non-null     float64
 6   liveness          50 non-null     float64
 7   valence           50 non-null     float64
 8   tempo             50 non-null     float64
 9   duration_ms       50 non-null     int64  
dtypes: float64(9), int64(1)
memory usage: 4.3 KB


Will make for loop to multiply all my 50 songs then get the weighted total of songs recommended

In [61]:
scaler = StandardScaler()
clean_user_df = scaler.fit_transform(clean_user_df)

In [66]:
clean_user_df = pd.DataFrame(clean_user_df)
clean_user_df

,0,1,2,3,4,5,6,7,8,9
0,0.317419,-0.036981,0.323278,0.845490,0.652558,-0.177262,-0.824492,-0.643703,-0.299331,-0.216192
1,-0.554238,0.317779,0.042514,-0.680076,1.667214,-0.177359,1.210497,0.608909,-0.820514,0.410117
2,-0.916485,0.532785,0.670162,-0.310913,0.377056,-0.171548,0.252350,1.555327,3.294907,1.523828
3,-1.324013,1.371308,0.028698,-0.952582,-0.767960,0.207868,-0.376881,-0.100905,-0.165444,1.381207
4,1.347559,-0.854004,0.507329,-0.427368,-0.569733,-0.175778,-0.446239,-2.195551,-0.384998,-0.691112
5,-2.093787,0.403781,1.016060,3.034852,-1.105955,-0.177359,-0.348279,-0.386222,-0.202496,2.565405
6,0.679666,0.059772,-0.202229,-0.663773,0.988537,-0.177200,-0.509162,0.003479,-0.209679,-0.299218
7,0.600424,0.479033,0.466868,-0.144381,2.480283,-0.177064,1.968434,1.499655,-0.123399,-1.436499
8,0.306099,-0.542245,-0.199762,-0.899013,-0.476331,-0.177359,0.910182,0.692416,-0.165619,0.212760
9,1.087194,1.048799,-0.499770,-0.320229,-0.880177,-0.177152,0.323854,1.123871,-0.077500,0.572091


In [67]:
column_rename_mapping = {
    '0': 'danceability',
    '1': 'energy',
    '2': 'loudness',
    '3': 'speechiness',
    '4': 'acousticness',
    '5': 'instrumentalness',
    '6': 'liveness',
    '7': 'valence',
    '8': 'tempo',
    '9': 'duration_ms',
}

In [76]:
clean_user_df.columns = clean_user_df.columns.astype(str)
clean_user_df.rename(columns=column_rename_mapping, inplace=True)

In [77]:
clean_user_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.317419,-0.036981,0.323278,0.845490,0.652558,-0.177262,-0.824492,-0.643703,-0.299331,-0.216192
1,-0.554238,0.317779,0.042514,-0.680076,1.667214,-0.177359,1.210497,0.608909,-0.820514,0.410117
2,-0.916485,0.532785,0.670162,-0.310913,0.377056,-0.171548,0.252350,1.555327,3.294907,1.523828
3,-1.324013,1.371308,0.028698,-0.952582,-0.767960,0.207868,-0.376881,-0.100905,-0.165444,1.381207
4,1.347559,-0.854004,0.507329,-0.427368,-0.569733,-0.175778,-0.446239,-2.195551,-0.384998,-0.691112
5,-2.093787,0.403781,1.016060,3.034852,-1.105955,-0.177359,-0.348279,-0.386222,-0.202496,2.565405
6,0.679666,0.059772,-0.202229,-0.663773,0.988537,-0.177200,-0.509162,0.003479,-0.209679,-0.299218
7,0.600424,0.479033,0.466868,-0.144381,2.480283,-0.177064,1.968434,1.499655,-0.123399,-1.436499
8,0.306099,-0.542245,-0.199762,-0.899013,-0.476331,-0.177359,0.910182,0.692416,-0.165619,0.212760
9,1.087194,1.048799,-0.499770,-0.320229,-0.880177,-0.177152,0.323854,1.123871,-0.077500,0.572091


In [78]:
# calculate median for all features
jose_median_reggaeton = clean_user_df.quantile(0.5)
jose_median_reggaeton

danceability        0.294778
energy              0.269402
loudness            0.114062
speechiness        -0.185141
acousticness       -0.244505
instrumentalness   -0.177108
liveness           -0.396902
valence             0.132220
tempo              -0.253892
duration_ms        -0.017928
Name: 0.5, dtype: float64

In [116]:
cleanest_spotify_df

,id,name,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,tempo,track_age_FLAG,era_FLAG,followers,genres,artist_name,artists_popularity,GMM_cluster,genres_reggaeton,KMeans_cluster
2981,4iozhXt27eMl39W5z7R8H6,Little Red Corvette,51,296907,0.748,0.746,-9.721,0.1110,0.28100,0.000033,...,123.144,39,1980s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,9
3021,04MjaNej8XUHI8jbz2THez,7,41,310467,0.747,0.749,-6.255,0.0563,0.09170,0.000000,...,100.722,29,1990s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,5
3052,7oZKxGPHuMVSVmFzuiKkWG,How Come U Don't Call Me Anymore,40,234773,0.556,0.128,-14.481,0.0688,0.90500,0.004060,...,83.065,28,1990s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,3
3065,5xcb3TD6lZ4X7RId59DNxo,She's Always in My Hair,41,207427,0.729,0.777,-10.157,0.0424,0.08910,0.000647,...,92.308,28,1990s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,5
3066,37boVGhMqQF129fn3jHQU4,Come,35,673160,0.750,0.843,-5.870,0.0798,0.00439,0.012200,...,91.378,27,1990s,5205880.0,"['funk', 'funk rock', 'minneapolis sound', 'mo...",Prince,76,3,False,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292963,0jngaMh4nKBzquXKeaaYzv,Bro,51,168259,0.708,0.622,-8.503,0.3770,0.17800,0.000000,...,173.133,1,2010s,7045.0,"['swedish gangsta rap', 'swedish hip hop', 'sw...",Lelo,46,3,False,5
292973,4UAJQSBKU7ARIuMnkwDp3n,Ahhä,52,172630,0.924,0.488,-7.217,0.1390,0.15500,0.000014,...,100.011,1,2010s,6097.0,"['swedish gangsta rap', 'swedish hip hop', 'sw...",La Rosy,46,3,False,9
292974,5I2QHXyUj8nx1Onu6fXu8C,Distans,51,177267,0.938,0.506,-6.851,0.1720,0.51000,0.000000,...,134.975,1,2010s,3354.0,"['swedish hip hop', 'swedish trap pop']",Sarettii,48,3,False,9
292976,7L9N5BTob9ISdb7IoxV2ZU,miSs U <3,50,202106,0.779,0.700,-7.917,0.0733,0.48400,0.000000,...,93.070,1,2010s,19310.0,"['swedish gangsta rap', 'swedish hip hop']",Alex Ceesay,55,3,False,9


In [117]:
df_clean= cleanest_spotify_df.loc[:,'duration_ms': 'tempo']
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3398 entries, 2981 to 292981
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   duration_ms       3398 non-null   int64  
 1   danceability      3398 non-null   float64
 2   energy            3398 non-null   float64
 3   loudness          3398 non-null   float64
 4   speechiness       3398 non-null   float64
 5   acousticness      3398 non-null   float64
 6   instrumentalness  3398 non-null   float64
 7   liveness          3398 non-null   float64
 8   valence           3398 non-null   float64
 9   tempo             3398 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 292.0 KB


In [118]:
df_clean = scaler.fit_transform(df_clean)
df_clean = pd.DataFrame(df_clean)

In [119]:
column_rename_mapping = {
    '0': 'duration_ms',
    '1': 'danceability',
    '2': 'energy',
    '3': 'loudness',
    '4': 'speechiness',
    '5': 'acousticness',
    '6': 'instrumentalness',
    '7': 'liveness',
    '8': 'valence',
    '9': 'tempo',
}

In [120]:
df_clean.columns = df_clean.columns.astype(str)

In [121]:
df_clean.rename(columns=column_rename_mapping, inplace=True)

In [123]:
cleanest_spotify_df.columns

Index(['id', 'name', 'popularity', 'duration_ms', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'track_age_FLAG', 'era_FLAG',
       'followers', 'genres', 'artist_name', 'artists_popularity',
       'GMM_cluster', 'genres_reggaeton', 'KMeans_cluster'],
      dtype='object')

In [131]:
df_clean

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_name,song_popularity,genres_reggaeton,artists_popularity,artist_name,era,genres
0,1.208973,0.416749,0.311663,-0.855617,-0.569301,0.373234,-0.244317,2.636485,0.760748,0.095870,Little Red Corvette,51,False,76,Prince,1980s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
1,1.408380,0.410781,0.328681,0.301205,-0.990408,-0.477498,-0.244552,-0.964018,0.132054,-0.652674,7,41,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
2,0.295259,-0.729045,-3.194073,-2.444327,-0.894177,3.177547,-0.215766,-0.799802,-0.345926,-1.242141,How Come U Don't Call Me Anymore,40,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
3,-0.106879,0.303363,0.487517,-1.001137,-1.097416,-0.489183,-0.239965,-0.948699,1.100932,-0.933570,She's Always in My Hair,41,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
4,6.741979,0.428684,0.861916,0.429703,-0.809493,-0.869877,-0.158053,1.049469,0.704769,-0.964617,Come,35,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,-0.682866,0.178042,-0.391753,-0.449094,1.478493,-0.089658,-0.244552,-0.734238,0.097605,1.764720,Bro,51,False,46,Lelo,2010s,"['swedish gangsta rap', 'swedish hip hop', 'sw..."
3394,-0.618588,1.467060,-1.151897,-0.019875,-0.353744,-0.193022,-0.244451,-0.604948,-0.841130,-0.676410,Ahhä,52,False,46,La Rosy,2010s,"['swedish gangsta rap', 'swedish hip hop', 'sw..."
3395,-0.550398,1.550607,-1.049788,0.102282,-0.099694,1.402381,-0.244552,-0.657032,0.446401,0.490840,Distans,51,False,48,Sarettii,2010s,"['swedish hip hop', 'swedish trap pop']"
3396,-0.185127,0.601747,0.050718,-0.253509,-0.859534,1.285534,-0.244552,-0.335339,0.510993,-0.908131,miSs U <3,50,False,55,Alex Ceesay,2010s,"['swedish gangsta rap', 'swedish hip hop']"


In [130]:
# add information that will come useful with final dataset
cleanest_spotify_df_reset = cleanest_spotify_df.reset_index(drop=True)
df_clean['track_name'] = cleanest_spotify_df_reset['name']
df_clean['song_popularity'] = cleanest_spotify_df_reset['popularity']
df_clean['genres_reggaeton'] = cleanest_spotify_df_reset['genres_reggaeton']
df_clean['artists_popularity'] = cleanest_spotify_df_reset['artists_popularity']
df_clean['artist_name'] = cleanest_spotify_df_reset['artist_name']
df_clean['era'] = cleanest_spotify_df_reset['era_FLAG']
df_clean['genres'] = cleanest_spotify_df_reset['genres']


In [94]:
user_stan_df = jose_median_reggaeton.copy().head(n)
user_stan_df

danceability        0.294778
energy              0.269402
loudness            0.114062
speechiness        -0.185141
acousticness       -0.244505
instrumentalness   -0.177108
liveness           -0.396902
valence             0.132220
tempo              -0.253892
duration_ms        -0.017928
Name: 0.5, dtype: float64

In [99]:
#reorder columns for both
column_order = df_stan.columns
column_order

Index(['duration_ms', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [103]:
user_stan_df= user_stan_df[column_order]

In [104]:
# both datasets have same order columnwise, now I can multiply
user_stan_df

duration_ms        -0.017928
danceability        0.294778
energy              0.269402
loudness            0.114062
speechiness        -0.185141
acousticness       -0.244505
instrumentalness   -0.177108
liveness           -0.396902
valence             0.132220
tempo              -0.253892
Name: 0.5, dtype: float64

In [108]:
#rename to match code
user_norm = user_stan_df

In [132]:
df_clean

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_name,song_popularity,genres_reggaeton,artists_popularity,artist_name,era,genres
0,1.208973,0.416749,0.311663,-0.855617,-0.569301,0.373234,-0.244317,2.636485,0.760748,0.095870,Little Red Corvette,51,False,76,Prince,1980s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
1,1.408380,0.410781,0.328681,0.301205,-0.990408,-0.477498,-0.244552,-0.964018,0.132054,-0.652674,7,41,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
2,0.295259,-0.729045,-3.194073,-2.444327,-0.894177,3.177547,-0.215766,-0.799802,-0.345926,-1.242141,How Come U Don't Call Me Anymore,40,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
3,-0.106879,0.303363,0.487517,-1.001137,-1.097416,-0.489183,-0.239965,-0.948699,1.100932,-0.933570,She's Always in My Hair,41,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
4,6.741979,0.428684,0.861916,0.429703,-0.809493,-0.869877,-0.158053,1.049469,0.704769,-0.964617,Come,35,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,-0.682866,0.178042,-0.391753,-0.449094,1.478493,-0.089658,-0.244552,-0.734238,0.097605,1.764720,Bro,51,False,46,Lelo,2010s,"['swedish gangsta rap', 'swedish hip hop', 'sw..."
3394,-0.618588,1.467060,-1.151897,-0.019875,-0.353744,-0.193022,-0.244451,-0.604948,-0.841130,-0.676410,Ahhä,52,False,46,La Rosy,2010s,"['swedish gangsta rap', 'swedish hip hop', 'sw..."
3395,-0.550398,1.550607,-1.049788,0.102282,-0.099694,1.402381,-0.244552,-0.657032,0.446401,0.490840,Distans,51,False,48,Sarettii,2010s,"['swedish hip hop', 'swedish trap pop']"
3396,-0.185127,0.601747,0.050718,-0.253509,-0.859534,1.285534,-0.244552,-0.335339,0.510993,-0.908131,miSs U <3,50,False,55,Alex Ceesay,2010s,"['swedish gangsta rap', 'swedish hip hop']"


In [135]:
import pandas as pd

# Assuming df_clean and user_norm are your DataFrames or Series
# Replace them with your actual DataFrames or Series.

# Matrix factorization
n = 3398

# Extract the values from the relevant columns of df_clean and user_norm
df_clean_values = df_clean.head(n).loc[:, 'duration_ms':'tempo'].values


In [136]:
# Convert user_norm to a DataFrame with a single row
user_norm_df = user_norm

In [150]:
# Perform element-wise multiplication
new_df = df_clean_values * user_norm_df.values
new_df = pd.DataFrame(new_df)

In [151]:
column_rename_mapping = {
    '0': 'duration_ms',
    '1': 'danceability',
    '2': 'energy',
    '3': 'loudness',
    '4': 'speechiness',
    '5': 'acousticness',
    '6': 'instrumentalness',
    '7': 'liveness',
    '8': 'valence',
    '9': 'tempo',
}

In [152]:
new_df.columns = new_df.columns.astype(str)
new_df.rename(columns=column_rename_mapping, inplace=True)

In [154]:
new_df.sum(axis=1)

0      -0.825222
1       1.127960
2      -1.345588
3       1.232902
4       0.598823
          ...   
3393   -0.444257
3394    0.587564
3395    0.109864
3396    0.484803
3397   -0.148273
Length: 3398, dtype: float64

In [155]:
# Calculate the 'total' column by summing along the columns
new_df['total'] = new_df.sum(axis=1)

# Display the resulting DataFrame
new_df.head()

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,total
0,-0.021674,0.122849,0.083963,-0.097594,0.105401,-0.091258,0.043271,-1.046425,0.100586,-0.024341,-0.825222
1,-0.025249,0.121089,0.088547,0.034356,0.183365,0.116751,0.043312,0.382620,0.017460,0.165708,1.127960
2,-0.005293,-0.214907,-0.860491,-0.278805,0.165549,-0.776927,0.038214,0.317443,-0.045738,0.315369,-1.345588
3,0.001916,0.089425,0.131338,-0.114192,0.203177,0.119608,0.042500,0.376540,0.145565,0.237026,1.232902
4,-0.120868,0.126367,0.232202,0.049013,0.149870,0.212690,0.027993,-0.416536,0.093185,0.244908,0.598823


### jose_playlist

In [156]:
# add important song features

# Reset index of cleanest_spotify_df to ensure alignment
cleanest_spotify_df_reset = cleanest_spotify_df.reset_index(drop=True)

# Add the new columns to new_df based on index matching
new_df['track_name'] = cleanest_spotify_df_reset['name']
new_df['song_popularity'] = cleanest_spotify_df_reset['popularity']
new_df['genres_reggaeton'] = cleanest_spotify_df_reset['genres_reggaeton']
new_df['artists_popularity'] = cleanest_spotify_df_reset['artists_popularity']
new_df['artist_name'] = cleanest_spotify_df_reset['artist_name']
new_df['era'] = cleanest_spotify_df_reset['era_FLAG']
new_df['genres'] = cleanest_spotify_df_reset['genres']


In [158]:
jose_playlist = new_df

In [160]:
jose_playlist.sort_values(by=['total'], ascending=False).head(10)[['track_name', 'song_popularity','artist_name', 'artists_popularity',  'era', 'genres' ,'total']]

,track_name,song_popularity,artist_name,artists_popularity,era,genres,total
703,Papi Chulo,57,El Chombo,56,2010s,"['latin', 'latin hip hop', 'panamanian pop', '...",1.626040
981,Voy A Beber (Original),46,Nicky Jam,88,2010s,"['latin', 'latin hip hop', 'reggaeton', 'trap ...",1.620245
1563,Home Is Where the Heart Is,44,Bliss n Eso,61,2010s,['australian hip hop'],1.613321
1280,Bitches,48,DJ Isaac,56,2010s,"['australian dance', 'classic hardstyle', 'eup...",1.611591
1842,Tu veux mon zizi,43,Francky Vincent,44,2000s,"['chanson paillarde', 'francoton', 'musique gu...",1.610589
688,Vivrant Thing,37,Q-Tip,66,1990s,"['alternative hip hop', 'east coast hip hop', ...",1.608482
2750,ФРЕЙМДАТА (PROD. BY CAKEBOY),43,GONE.Fludd,67,2010s,"['russian hip hop', 'russian trap']",1.604547
689,Vivrant Thing,41,Q-Tip,66,1990s,"['alternative hip hop', 'east coast hip hop', ...",1.602625
1916,Хлоп-Хлоп,25,GUF,59,2000s,['russian hip hop'],1.595097
2203,Opos Tote,18,Stixoima,36,2000s,['greek hip hop'],1.593327


In [162]:
jose_playlist['song_popularity_score'] = jose_playlist['total'] * jose_playlist['song_popularity']
jose_playlist['artist_popularity_score'] = jose_playlist['total'] * jose_playlist['artists_popularity']


In [163]:
jose_playlist.sort_values(by=['song_popularity_score'], ascending=False).head(10)[['track_name', 'song_popularity','artist_name', 'artists_popularity',  'era', 'genres' ,'total']]

,track_name,song_popularity,artist_name,artists_popularity,era,genres,total
1142,Everybody,66,Logic,83,2010s,"['conscious hip hop', 'dmv rap', 'hip hop', 'p...",1.573545
932,Riot,69,Hollywood Undead,75,2010s,"['alternative metal', 'modern rock', 'nu metal...",1.487320
3099,May I,73,Flo Milli,74,2010s,"['dance pop', 'pop', 'pop rap', 'r&b', 'viral ...",1.399483
1152,All About That Bass,74,Meghan Trainor,80,2010s,"['dance pop', 'hip pop', 'pop', 'post-teen pop']",1.369852
1369,Batman,80,LPB Poody,67,2010s,['florida rap'],1.241419
896,Crazy Bitch,69,Buckcherry,61,2000s,"['alternative metal', 'glam metal', 'hard rock...",1.416520
2995,Tequila,64,Mario Bautista,62,2010s,"['latin', 'latin pop', 'latin viral pop', 'mex...",1.508087
761,Sugar,82,Maroon 5,91,2010s,"['pop', 'pop rock']",1.163910
978,Fuck You,64,Lily Allen,72,2000s,"['dance pop', 'electropop', 'pop']",1.480873
703,Papi Chulo,57,El Chombo,56,2010s,"['latin', 'latin hip hop', 'panamanian pop', '...",1.626040


In [164]:
jose_playlist.sort_values(by=['artist_popularity_score'], ascending=False).head(10)[['track_name', 'song_popularity','artist_name', 'artists_popularity',  'era', 'genres' ,'total']]

,track_name,song_popularity,artist_name,artists_popularity,era,genres,total
981,Voy A Beber (Original),46,Nicky Jam,88,2010s,"['latin', 'latin hip hop', 'reggaeton', 'trap ...",1.620245
1142,Everybody,66,Logic,83,2010s,"['conscious hip hop', 'dmv rap', 'hip hop', 'p...",1.573545
538,Hard Knock Life (Ghetto Anthem),51,JAY-Z,87,2010s,"['east coast hip hop', 'hip hop', 'pop rap', '...",1.499707
785,"Wanksta - From ""8 Mile"" Soundtrack",57,50 Cent,85,2000s,"['east coast hip hop', 'gangster rap', 'hip ho...",1.508589
534,Never Change,46,JAY-Z,87,2000s,"['east coast hip hop', 'hip hop', 'pop rap', '...",1.469635
820,Donde Mi No Vengas / Nunca Me Quedo Atras / Ya...,29,Daddy Yankee,91,1990s,"['latin', 'latin hip hop', 'reggaeton', 'trap ...",1.390653
301,Kill the DJ,56,Green Day,83,2010s,"['permanent wave', 'pop punk', 'punk']",1.516740
1004,Pound The Alarm,67,Nicki Minaj,90,2010s,"['dance pop', 'hip pop', 'pop', 'pop rap', 'po...",1.295536
251,I Wonder If Heaven Got A Ghetto - Hip-Hop Version,43,2Pac,83,1990s,"['g funk', 'gangster rap', 'hip hop', 'rap', '...",1.389870
505,So Bad,59,Eminem,94,2010s,"['detroit hip hop', 'hip hop', 'rap']",1.214583


In [167]:
jose_playlist.sort_values(
    by=['genres_reggaeton', 'artist_popularity_score'],
    ascending=[False, False]
).head(10)[['track_name', 'song_popularity', 'artist_name', 'artists_popularity', 'era', 'genres', 'total']]


,track_name,song_popularity,artist_name,artists_popularity,era,genres,total
981,Voy A Beber (Original),46,Nicky Jam,88,2010s,"['latin', 'latin hip hop', 'reggaeton', 'trap ...",1.620245
820,Donde Mi No Vengas / Nunca Me Quedo Atras / Ya...,29,Daddy Yankee,91,1990s,"['latin', 'latin hip hop', 'reggaeton', 'trap ...",1.390653
1205,Vete,79,Bad Bunny,98,2010s,"['latin', 'reggaeton', 'trap latino']",1.145446
1209,La Zona,71,Bad Bunny,98,2010s,"['latin', 'reggaeton', 'trap latino']",0.969114
2995,Tequila,64,Mario Bautista,62,2010s,"['latin', 'latin pop', 'latin viral pop', 'mex...",1.508087
703,Papi Chulo,57,El Chombo,56,2010s,"['latin', 'latin hip hop', 'panamanian pop', '...",1.626040
799,Güasa Güasa (feat. Voltio),48,Tego Calderon,73,2000s,"['latin', 'latin hip hop', 'rap boricua', 'reg...",1.191434
797,Punto y Aparte,65,Tego Calderon,73,2000s,"['latin', 'latin hip hop', 'rap boricua', 'reg...",1.185508
697,Mueve el Cuerpo y la Cabeza,28,El Chombo,56,1990s,"['latin', 'latin hip hop', 'panamanian pop', '...",1.467411
1356,¿Que Somos?,0,Sech,89,2010s,"['latin', 'panamanian pop', 'reggaeton', 'trap...",0.917733


### Eucledian distances

In [173]:
#dataset df
df_clean.head(n).loc[:, 'duration_ms':'tempo']

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1.208973,0.416749,0.311663,-0.855617,-0.569301,0.373234,-0.244317,2.636485,0.760748,0.095870
1,1.408380,0.410781,0.328681,0.301205,-0.990408,-0.477498,-0.244552,-0.964018,0.132054,-0.652674
2,0.295259,-0.729045,-3.194073,-2.444327,-0.894177,3.177547,-0.215766,-0.799802,-0.345926,-1.242141
3,-0.106879,0.303363,0.487517,-1.001137,-1.097416,-0.489183,-0.239965,-0.948699,1.100932,-0.933570
4,6.741979,0.428684,0.861916,0.429703,-0.809493,-0.869877,-0.158053,1.049469,0.704769,-0.964617
...,...,...,...,...,...,...,...,...,...,...
3393,-0.682866,0.178042,-0.391753,-0.449094,1.478493,-0.089658,-0.244552,-0.734238,0.097605,1.764720
3394,-0.618588,1.467060,-1.151897,-0.019875,-0.353744,-0.193022,-0.244451,-0.604948,-0.841130,-0.676410
3395,-0.550398,1.550607,-1.049788,0.102282,-0.099694,1.402381,-0.244552,-0.657032,0.446401,0.490840
3396,-0.185127,0.601747,0.050718,-0.253509,-0.859534,1.285534,-0.244552,-0.335339,0.510993,-0.908131


In [178]:
pd.DataFrame([reference_song])

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0.5,-0.017928,0.294778,0.269402,0.114062,-0.185141,-0.244505,-0.177108,-0.396902,0.13222,-0.253892


In [180]:
import numpy as np
import pandas as pd

# dataset from cluster
songs_df = df_clean.head(n).loc[:, 'duration_ms':'tempo']

# user song information
reference_song = user_norm_df

# Convert the reference song to a DataFrame with one row
reference_song_df = pd.DataFrame([reference_song])

# Calculate Euclidean distance for each song in the dataset
distances = np.linalg.norm(songs_df.values - reference_song_df.values, axis=1)

# Add distances as a new column in the dataset
songs_df['euclidean_distance'] = distances

# Display the resulting DataFrame
print(songs_df.head())


   duration_ms  danceability    energy  loudness  speechiness  acousticness  \
0     1.208973      0.416749  0.311663 -0.855617    -0.569301      0.373234   
1     1.408380      0.410781  0.328681  0.301205    -0.990408     -0.477498   
2     0.295259     -0.729045 -3.194073 -2.444327    -0.894177      3.177547   
3    -0.106879      0.303363  0.487517 -1.001137    -1.097416     -0.489183   
4     6.741979      0.428684  0.861916  0.429703    -0.809493     -0.869877   

   instrumentalness  liveness   valence     tempo  euclidean_distance  
0         -0.244317  2.636485  0.760748  0.095870            3.565773  
1         -0.244552 -0.964018  0.132054 -0.652674            1.809499  
2         -0.215766 -0.799802 -0.345926 -1.242141            5.767979  
3         -0.239965 -0.948699  1.100932 -0.933570            1.974877  
4         -0.158053  1.049469  0.704769 -0.964617            7.061953  


In [181]:
songs_df

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,euclidean_distance
0,1.208973,0.416749,0.311663,-0.855617,-0.569301,0.373234,-0.244317,2.636485,0.760748,0.095870,3.565773
1,1.408380,0.410781,0.328681,0.301205,-0.990408,-0.477498,-0.244552,-0.964018,0.132054,-0.652674,1.809499
2,0.295259,-0.729045,-3.194073,-2.444327,-0.894177,3.177547,-0.215766,-0.799802,-0.345926,-1.242141,5.767979
3,-0.106879,0.303363,0.487517,-1.001137,-1.097416,-0.489183,-0.239965,-0.948699,1.100932,-0.933570,1.974877
4,6.741979,0.428684,0.861916,0.429703,-0.809493,-0.869877,-0.158053,1.049469,0.704769,-0.964617,7.061953
...,...,...,...,...,...,...,...,...,...,...,...
3393,-0.682866,0.178042,-0.391753,-0.449094,1.478493,-0.089658,-0.244552,-0.734238,0.097605,1.764720,2.862872
3394,-0.618588,1.467060,-1.151897,-0.019875,-0.353744,-0.193022,-0.244451,-0.604948,-0.841130,-0.676410,2.231120
3395,-0.550398,1.550607,-1.049788,0.102282,-0.099694,1.402381,-0.244552,-0.657032,0.446401,0.490840,2.654453
3396,-0.185127,0.601747,0.050718,-0.253509,-0.859534,1.285534,-0.244552,-0.335339,0.510993,-0.908131,1.918536


In [182]:
songs_df['euclidean_distance']

0       3.565773
1       1.809499
2       5.767979
3       1.974877
4       7.061953
          ...   
3393    2.862872
3394    2.231120
3395    2.654453
3396    1.918536
3397    3.054870
Name: euclidean_distance, Length: 3398, dtype: float64

In [185]:
#add eucledian distance column
jose_playlist['euclidean_distance'] = songs_df['euclidean_distance']


In [184]:
jose_playlist

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,track_name,song_popularity,genres_reggaeton,artists_popularity,artist_name,era,genres,song_popularity_score,artist_popularity_score,euclidean_distance
0,-0.021674,0.122849,0.083963,-0.097594,0.105401,-0.091258,0.043271,-1.046425,0.100586,-0.024341,...,Little Red Corvette,51,False,76,Prince,1980s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",-42.086304,-62.716844,3.565773
1,-0.025249,0.121089,0.088547,0.034356,0.183365,0.116751,0.043312,0.382620,0.017460,0.165708,...,7,41,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",46.246377,85.724992,1.809499
2,-0.005293,-0.214907,-0.860491,-0.278805,0.165549,-0.776927,0.038214,0.317443,-0.045738,0.315369,...,How Come U Don't Call Me Anymore,40,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",-53.823506,-102.264662,5.767979
3,0.001916,0.089425,0.131338,-0.114192,0.203177,0.119608,0.042500,0.376540,0.145565,0.237026,...,She's Always in My Hair,41,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",50.548996,93.700579,1.974877
4,-0.120868,0.126367,0.232202,0.049013,0.149870,0.212690,0.027993,-0.416536,0.093185,0.244908,...,Come,35,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",20.958796,45.510529,7.061953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,0.012242,0.052483,-0.105539,-0.051225,-0.273729,0.021922,0.043312,0.291420,0.012905,-0.448048,...,Bro,51,False,46,Lelo,2010s,"['swedish gangsta rap', 'swedish hip hop', 'sw...",-22.657089,-20.435805,2.862872
3394,0.011090,0.432458,-0.310324,-0.002267,0.065492,0.047195,0.043294,0.240105,-0.111214,0.171735,...,Ahhä,52,False,46,La Rosy,2010s,"['swedish gangsta rap', 'swedish hip hop', 'sw...",30.553304,27.027923,2.231120
3395,0.009867,0.457086,-0.282815,0.011667,0.018457,-0.342889,0.043312,0.260777,0.059023,-0.124620,...,Distans,51,False,48,Sarettii,2010s,"['swedish hip hop', 'swedish trap pop']",5.603070,5.273478,2.654453
3396,0.003319,0.177382,0.013664,-0.028916,0.159135,-0.314320,0.043312,0.133097,0.067564,0.230567,...,miSs U <3,50,False,55,Alex Ceesay,2010s,"['swedish gangsta rap', 'swedish hip hop']",24.240138,26.664152,1.918536


In [187]:
jose_playlist.sort_values(by=['euclidean_distance'], ascending=True).head(10)[['track_name', 'song_popularity','artist_name', 'artists_popularity',  'era', 'genres' ,'euclidean_distance']]

,track_name,song_popularity,artist_name,artists_popularity,era,genres,euclidean_distance
3194,Club-Banger,42,Svabo,50,2010s,[],0.925338
2769,Нужен Xanax,45,Pasha Technique,42,2010s,"['russian hip hop', 'russian trap']",1.014779
1273,Jump Out The Window,66,Big Sean,85,2010s,"['detroit hip hop', 'hip hop', 'pop rap', 'rap...",1.017024
3077,Ne fuis pas,57,2TH,53,2010s,['rap calme'],1.039141
2795,Sad Song,52,MARUV,55,2010s,"['russian dance pop', 'russian pop', 'ukrainia...",1.083650
84,Sure Shot - Remastered 2009,47,Beastie Boys,73,1990s,"['alternative rock', 'east coast hip hop', 'hi...",1.095552
3174,Room For Rent,41,Seth Sentry,51,2010s,"['australian hip hop', 'australian indie', 'au...",1.105206
1467,Corrida,72,SCH,83,2010s,"['pop urbaine', 'rap marseille']",1.126880
968,Leader Of The Delinquents,4,Kid Cudi,87,2010s,"['hip hop', 'ohio hip hop', 'rap']",1.139494
29,"Amor, Amor - Versão Estendida",28,Barão Vermelho,62,1980s,"['brazilian rock', 'mpb', 'rock nacional brasi...",1.142156


In [199]:
jose_playlist['song_popularity_score_ED'] = jose_playlist['song_popularity']/jose_playlist['euclidean_distance'] 
jose_playlist['artist_popularity_score_ED'] = jose_playlist['artists_popularity'] /jose_playlist['euclidean_distance']

In [200]:
jose_playlist.sort_values(by=['song_popularity_score_ED'], ascending=False).head(10)[['track_name', 'song_popularity','artist_name', 'artists_popularity',  'era', 'genres' ,'euclidean_distance']]

,track_name,song_popularity,artist_name,artists_popularity,era,genres,euclidean_distance
1106,"thank u, next",84,Ariana Grande,95,2010s,"['pop', 'post-teen pop']",1.264542
1273,Jump Out The Window,66,Big Sean,85,2010s,"['detroit hip hop', 'hip hop', 'pop rap', 'rap...",1.017024
1467,Corrida,72,SCH,83,2010s,"['pop urbaine', 'rap marseille']",1.126880
1122,White Girl,69,Shy Glizzy,67,2010s,"['dmv rap', 'drill', 'rap', 'southern hip hop'...",1.166283
3077,Ne fuis pas,57,2TH,53,2010s,['rap calme'],1.039141
1311,AP - Music from the film Boogie,83,Pop Smoke,92,2010s,['brooklyn drill'],1.548644
1347,Goosebumps,89,HVME,81,2010s,['slap house'],1.666271
1367,BOYSHIT,70,Madison Beer,81,2010s,"['alt z', 'dance pop', 'electropop', 'pop', 'p...",1.319283
3097,Paramedic!,68,SOB X RBE,62,2010s,"['cali rap', 'trap', 'vapor trap']",1.295346
1576,OMG,66,Capo Plaza,77,2010s,"['italian hip hop', 'trap italiana']",1.288869


In [196]:
jose_playlist.sort_values(by=['artist_popularity_score_ED'], ascending=False).head(10)[['track_name', 'song_popularity','artist_name', 'artists_popularity',  'era', 'genres' ,'euclidean_distance']]

,track_name,song_popularity,artist_name,artists_popularity,era,genres,euclidean_distance
957,Worst Behavior,57,Drake,98,2010s,"['canadian hip hop', 'canadian pop', 'hip hop'...",1.155635
1273,Jump Out The Window,66,Big Sean,85,2010s,"['detroit hip hop', 'hip hop', 'pop rap', 'rap...",1.017024
964,Worst Behavior,59,Drake,98,2010s,"['canadian hip hop', 'canadian pop', 'hip hop'...",1.251042
968,Leader Of The Delinquents,4,Kid Cudi,87,2010s,"['hip hop', 'ohio hip hop', 'rap']",1.139494
1106,"thank u, next",84,Ariana Grande,95,2010s,"['pop', 'post-teen pop']",1.264542
1467,Corrida,72,SCH,83,2010s,"['pop urbaine', 'rap marseille']",1.126880
508,Cum On Everybody,47,Eminem,94,1990s,"['detroit hip hop', 'hip hop', 'rap']",1.291593
499,Cum On Everybody,50,Eminem,94,1990s,"['detroit hip hop', 'hip hop', 'rap']",1.300770
84,Sure Shot - Remastered 2009,47,Beastie Boys,73,1990s,"['alternative rock', 'east coast hip hop', 'hi...",1.095552
1059,Rockin’,68,The Weeknd,96,2010s,"['canadian contemporary r&b', 'canadian pop', ...",1.482432


In [201]:
jose_playlist

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,genres_reggaeton,artists_popularity,artist_name,era,genres,song_popularity_score,artist_popularity_score,euclidean_distance,song_popularity_score_ED,artist_popularity_score_ED
0,-0.021674,0.122849,0.083963,-0.097594,0.105401,-0.091258,0.043271,-1.046425,0.100586,-0.024341,...,False,76,Prince,1980s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",-42.086304,-62.716844,3.565773,14.302650,21.313753
1,-0.025249,0.121089,0.088547,0.034356,0.183365,0.116751,0.043312,0.382620,0.017460,0.165708,...,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",46.246377,85.724992,1.809499,22.658210,42.000585
2,-0.005293,-0.214907,-0.860491,-0.278805,0.165549,-0.776927,0.038214,0.317443,-0.045738,0.315369,...,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",-53.823506,-102.264662,5.767979,6.934838,13.176193
3,0.001916,0.089425,0.131338,-0.114192,0.203177,0.119608,0.042500,0.376540,0.145565,0.237026,...,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",50.548996,93.700579,1.974877,20.760782,38.483400
4,-0.120868,0.126367,0.232202,0.049013,0.149870,0.212690,0.027993,-0.416536,0.093185,0.244908,...,False,76,Prince,1990s,"['funk', 'funk rock', 'minneapolis sound', 'mo...",20.958796,45.510529,7.061953,4.956136,10.761896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,0.012242,0.052483,-0.105539,-0.051225,-0.273729,0.021922,0.043312,0.291420,0.012905,-0.448048,...,False,46,Lelo,2010s,"['swedish gangsta rap', 'swedish hip hop', 'sw...",-22.657089,-20.435805,2.862872,17.814281,16.067783
3394,0.011090,0.432458,-0.310324,-0.002267,0.065492,0.047195,0.043294,0.240105,-0.111214,0.171735,...,False,46,La Rosy,2010s,"['swedish gangsta rap', 'swedish hip hop', 'sw...",30.553304,27.027923,2.231120,23.306681,20.617448
3395,0.009867,0.457086,-0.282815,0.011667,0.018457,-0.342889,0.043312,0.260777,0.059023,-0.124620,...,False,48,Sarettii,2010s,"['swedish hip hop', 'swedish trap pop']",5.603070,5.273478,2.654453,19.212999,18.082823
3396,0.003319,0.177382,0.013664,-0.028916,0.159135,-0.314320,0.043312,0.133097,0.067564,0.230567,...,False,55,Alex Ceesay,2010s,"['swedish gangsta rap', 'swedish hip hop']",24.240138,26.664152,1.918536,26.061540,28.667694


Eucledian distances were not great

In [202]:
jose_playlist.to_csv("../../Reggaeton_Recommender_System/Output/jose_playlist.csv", index=False)